In [1]:
import openai
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from the .env file
load_dotenv("keys.env")

True

In [3]:
# Get the API key from the environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Check if the API key is loaded correctly
if OPENAI_API_KEY is None:
    raise ValueError("API key is not set. Please check your .env file.")

In [5]:
client = openai.Client(api_key=OPENAI_API_KEY)

In [12]:
# Define the Assistant with capabilities

assistant = client.beta.assistants.create(
    name="Personal AI Assistant",
    instructions="You are a personal assistant capable of managing tasks such as writing emails, reading PDF files, scheduling meetings, and conducting searches. Use the tools provided when necessary.",
    tools=[
        {"type": "code_interpreter"},
        {"type": "file_search"},
        # {"type": "function"}
    ],
    model="gpt-4o"
)


In [13]:
assistant = client.beta.assistants.create(
  instructions="You are a weather bot. Use the provided functions to answer questions.",
  model="gpt-4o",
  tools=[
    {
      "type": "function",
      "function": {
        "name": "get_current_temperature",
        "description": "Get the current temperature for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            },
            "unit": {
              "type": "string",
              "enum": ["Celsius", "Fahrenheit"],
              "description": "The temperature unit to use. Infer this from the user's location."
            }
          },
          "required": ["location", "unit"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "get_rain_probability",
        "description": "Get the probability of rain for a specific location",
        "parameters": {
          "type": "object",
          "properties": {
            "location": {
              "type": "string",
              "description": "The city and state, e.g., San Francisco, CA"
            }
          },
          "required": ["location"]
        }
      }
    }
  ]
)



In [14]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What's the weather in San Francisco today and the likelihood it'll rain?",
)

In [15]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)
 
# Define the list to store tool outputs
tool_outputs = []
 
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  if tool.function.name == "get_current_temperature":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "57"
    })
  elif tool.function.name == "get_rain_probability":
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": "0.06"
    })
 
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")
 
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

requires_action
Tool outputs submitted successfully.
SyncCursorPage[Message](data=[Message(id='msg_lpYXZnI7N5p7VA0gyOdPajIV', assistant_id='asst_3R1H436806X12RnnABQDKc5N', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The current temperature in San Francisco is 57°F, and there is a 6% chance of rain today.'), type='text')], created_at=1730135729, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_ieI9a4uojIdiV29XB1k2M7HM', status=None, thread_id='thread_wu8STYZDy9Llys3ifngAuh3n'), Message(id='msg_Pc6FWcu8ZlErzCSpQZkMICa6', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="What's the weather in San Francisco today and the likelihood it'll rain?"), type='text')], created_at=1730135621, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='t